**Вопросы для повторения**:

1. для чего вводили рэнжи?
2. какие билиотеки рэнжей мы смотрели?
3. чем view отличается от action?
4. какая здесь проблема?

```c++
auto make_some_event_numbers()
{
    std::vector<int> v = /* some numbers here */;
    return v | rv::filter(is_even);
}
```

5. что делает функция? как называется фича?

```c++
struct Person
{
  std::string surname;
  std::string name;
};

void f(std::vector<Person>& people)
{
    rs::sort(people, rg::less{}, &Person::name);
}
```

6. что делает этот код и в чём его проблема? как чинить?

```c++
for (const auto& person : people_ids | rv::transform(get_person_from_database_by_id)
                                     | rv::filter(is_student))
{
     std::cout << person.name << ", stay home" << std::endl;
}
```

7. Какие плюсы и минусы ренжей, когда их использовать скорее следует?

8. Работает ли код? Какие ошибки?

```c++
using Task = std::function<void>;

class TasksExecutor
{
private:
    bool is_stopped = false;    
    std::queue<Task> tasks;
    
public:
    TasksExecutor()
    {
        std::thread([this](){
            while (!is_stopped) { /* process tasks from queue */ }
        }).detach();
    }
    
    ~TasksExecutor() {
        is_stopped = true;
    }
    
    void stop() { is_stopped = true; }

    void push_task(Task&& task) { /* push task to queue */ }
};
```

<details>
<summary>подсказка</summary>
<p>

* race condition по `is_stopped`
* захват `this` + `detach` - хождение по разрушенному объекту после отработки `~TasksExecutor()`

</p>
</details>

<br />

### Мелкие нововведения C++11 / C++14 / C++17 / C++20

<br />

##### auto (since C++11)

https://en.cppreference.com/w/cpp/language/auto

`auto` - синтаксический сахар, указывающий компилятору вывести тип самостоятельно.

По каким правилам компилятор выводит тип?

> in the type specifier of a variable: `auto x = expr;`. The type is deduced from the initializer. If the placeholder type specifier is auto, the variable type is deduced from the initializer using the rules for template argument deduction from a function call (see template argument deduction#Other contexts for details). 

Компилятор используется правила вывода типов у аргументов шаблонов:

https://en.cppreference.com/w/cpp/language/template_argument_deduction

**Упрощённо**: у типа выражения убираются квалификаторы `const`, `&` и `&&` (более строгое определение - по ссылке)

**Упражнение**:
    
Какой тип будет у переменной?

```c++
int x;

// 1
auto y = x;  // ???

// 2:
int& a = x;
auto y = a;  // ???

// 3:
const int& a = x;
auto y = a;  // ???

// 4:
const std::string& get_name();
auto name        = get_name();  // ???
auto& name       = get_name();  // ???
const auto& name = get_name();  // ???

// 5:
std::string& get_name();
auto name        = get_name();  // ???
auto& name       = get_name();  // ???
const auto& name = get_name();  // ???

// 6:
std::string get_name();
auto name        = get_name();  // ???
auto& name       = get_name();  // ???
const auto& name = get_name();  // ???

// 7:
auto v       = std::vector<int>{1, 2, 3};  // ???
const auto v = std::vector<int>{1, 2, 3};  // ???

// 8:
auto v = {1, 2, 3};  // ???
```

**Рекомендации по использованию:**

По вопросу использования `auto` мир С++ раскололся на две секты:
* google coding guidelines - минимизация `auto`
* Herb Sutter (и многие другие) - AAA (almost always auto)

Разберём, какие аргументы стоят за этими спорами.

__Google coding guidelines__ рекомендуют использовать `auto` только там, где:
* тип очевиден по выражению справа (прямо по тексту справа)
* тип - итератор коллекции

```c++
auto processor = std::make_unique<LogsProcessor>();  // тип очевиден по выражению справа
auto dog = dynamic_cast<Dog>(animal);                // тип очевиден по выражению справа
auto it = std::begin(v);                             // итератор, чтобы не писать std::vector<int>::iterator

std::string name = get_name();  // из выражения справа не очевиден тип, надо подглядывать в определение get_name()
std::string full_name = name + " " + surname;  // из выражения справа не очевиден тип, надо смотреть типы name, surname
```

**Almost always auto**
* Пишите `auto` везде где это помогает скоратить код или подняться на уровень абстракции

```c++
auto logs_processor = create_logs_processor();
auto logs = logs_processor->run("logs.txt");
auto request_future = send_logs_to_server(std::move(logs));
```

Идея, стоящая за подходом: в первую очередь важна логика алгоритма, тип объектов вторичен. Логика алгоритма - более высокий уровень абстракции кода.

Сравните:

```c++
std::shared_ptr<LogsProcessor> logs_processor = create_logs_processor();
std::vector<LogRecord> logs = logs_processor->run("logs.txt");
std::future<LogsReceiveResponse> request_future = send_logs_to_server(std::move(logs));
```

Замечания:

* Конечно, когда дело доходит до специфики С++, тип объектов важен. И тут на помощь приходят нормальные IDE. Т.е. задача красиво показать читателю тип - это задача IDE, а текст отражает более высокий уровень абстракции.
* Всякие `int` и особенно `bool`, скорее всего, не стоит заменять на `auto`.

**Кто лучше?**

* Almost always auto - код значительно меньше по объёму и алгоритм проще для чтения, но нужна нормальная IDE.
* Google coding guidelines - код читать тяжело, но глазами видно ошибки с типами.

**Замечание**: Google работает с монорепозиторием (много, очень много кода в одном git-репозитории), который ни в одну IDE-шку не влезет. Поэтому его guidelines для самого Google логичны. Если кто-то на вполне нормальном проекте выбирает подход google coding guidelines для `auto` просто потому что Google умный и ему виднее, то так делать не надо.

**raw pointers**

Отдельная рекомендация, для raw pointers использовать `auto*`. Это связано с частым нарушением ошибок владения в языке С++.

```c++
std::unique_ptr<LogsProcessor> create_logs_processor();
LogsProcessor* create_logs_processor();  // да, это плохой С++, но его уже много

auto logs_processor = create_logs_processor();  // for shared_ptr
auto* logs_processor = create_logs_processor();  // for raw pointer


auto* network_subsystem = NetworkSubsystem::find(app_context);
if (network_subsystem)
    network_subsystem->send_request(r);
```

<br />

##### auto return type

auto return type - просьба компилятору самостоятельно вывести тип возвращаемого значения:

```c++
auto create_logs_processor()
{
    const auto settings = read_log_settings();
    return std::make_shared<LogsProcessor>(settings);
}
// компилятор автоматически выведет std::shared_ptr<LogsProcessor>
```

**Замечание 1:** auto return type можно использовать только в тех случаях, когда компилятор может вывести тип. В примере выше - можно. А вот так - нельзя. Почему?

```c++
// LogsProcessor.h
auto create_logs_processor();

// LogsProcessor.cpp
auto create_logs_processor()
{
    const auto settings = read_log_settings();
    return std::make_shared<LogsProcessor>(settings);
}
```

**Замечание 2**: споры, использовать или нет auto return type - аналогичны спорам, использовать или нет `auto`. Если у вас хорошая IDE, которая всё нужное подсветит, почему бы и нет. Если вы работаете в Google и программируете в блокноте, то, наверное, не стоит.

<br />

##### auto + trailing return type

Конструкция вида:
    
```c++
auto create_logs_processor() -> std::shared_ptr<LogsProcessor>;
```

Мне сложно сказать, почему её любят использовать и resharper, например, усиленно рекомендует писать именно в таком стиле.

Есть одно преимущество: для методов, возвращающих объект с типом из пространства класса, можно не указывать класс. Пример:

```c++
class LogsProcessor
{
public:
    struct ProcessedItem { /*...*/ };
    
    ProcessedItem* last_processed() const;
};

// standard:
LogsProcessor::ProcessedItem* LogsProcessor::last_processed() const
{ /*...*/ }

// trailing return type:
auto LogsProcessor::last_processed() const -> ProcessedItem*
{ /*...*/ }
```

Не нахожу рациональным рекомендовать вам новую непривычную конструкцию, которая чаще добавляет объём кода без особых улучшений. Не знаю, почему resharper рекомендует такой подход.

Бывает хитрая шаблонная магия, где без trialing return type нельзя:

```c++
std::shared_ptr<NetworkSubsystem> create_network();
std::unique_ptr<ResourcesSubsystem> create_resources();

template<typename Factory>
auto create_subsystem(const Factory& factory, const char* name) -> decltype(factory())
{
    if (is_subsystem_enabled(name))
        return factory();  // std::shared_ptr / std::unqiue_ptr
    
    return nullptr;  // std::nullptr_t
}
```

Без `-> decltype(factory())` код не скомпилируется, т.к. у разных `return` разный тип, компилятор автоматически тип не выведет.

Закинуть этот код на [godbot.org](https://godbolt.org) для демонстрации, показать:

```c++
#include <memory>

auto f1()
{
    if (true)
        return std::make_unique<int>(0);
    return nullptr;
}

auto f2() -> decltype(std::make_unique<int>(0))
{
    if (true)
        return std::make_unique<int>(0);
    return nullptr;
}
```

<br />

##### C++11: user-defined literals / C++14: user-defined literals for STL

https://en.cppreference.com/w/cpp/language/user_literal

https://www.modernescpp.com/index.php/user-defined-literals

> Allows integer, floating-point, character, and string literals to produce objects of user-defined type by defining a user-defined suffix.

Про user-defined literal говорят, что это способ совместить значеие и единицы измерения.

Например, такой код:

```c++
const float distance_1_m = 12.3f * 1000; // 12.3 km
const float distance_2_m = 3.f / 100; // 3 cm
const float distance_3_m = 45.f / 1000; // 45 mm
```

Хотелось бы писать вот так:

```c++
const float distance_1_m = 12.3_km;
const float distance_2_m = 3_cm;
const float distance_3_m = 45_mm;
```

И это возможно. Достаточно определить оператор, который нужно применяемый к литералу, когда к нему приписан префикс `_km` / `_cm` / `_mm`:

```c++
constexpr double operator"" _km(double value)
{
    return value * 1000.0;
}

constexpr double operator"" _m(double value)
{
    return value;
}

constexpr double operator"" _mm(double value)
{
    return value / 1000.0;
}

constexpr double operator"" _cm(double value)
{
    return value / 100.0;
}

constexpr double operator"" _inch(double value)
{
    return value * 0.0254;
}
```

Ошибки единиц измерения, когда в программах сантиметры складывают с метрами, а градусы с радианами - довольно частое явление. user-defined literals помогают избежать части из них.

Такой код будет работать корректно:

```c++
const auto distance = 12.3_km + 200_m;
```

<br />

Чтобы ещё снизить вероятность ошибки, иногда делают следующий шаг: для представления расстояния `double` заменяется на специальный тип `Distance`:

```c++
class Distance
{
    double value_m;
    
    constexpr explicit Distance(const double i_value_m) : value_m(i_value_m) {}

public:
    constexpr static Distance from_meters(const double m) { return Distance(m); }
    constexpr double to_meters() const noexcept { return value_m; }
    constexpr friend Distance operator +(const Distance l, const Distance r) noexcept;
    constexpr friend Distance operator *(const Distance l, const double r) noexcept;
    // заметьте:
    //   + только между двумя Distance, double + Distance - такого оператора нет
    //   * только между Distance и double, нельзя сделать Distance * Distance и получить Distance
    ...;
};

constexpr Distance operator"" _km(double value)
{
    return Distance::from_meters(value * 1000.0);
}

constexpr Distance operator"" _m(double value)
{
    return Distance::from_meters(value);
}
```

Тогда:

```c++
12.3_km + 200_m; // ok

12.3_km + 200; // compile-time error:
               //   с точки зрения компилятора не определён operator+ (Distance, double)
               //   с точки зрения программиста, непонятны единицы измерения у числа 200
```

<br />

В стандартной библиотеке C++ имеются свои user-defined literals для некоторых типов.

Например, для единиц измерения интервала времени:

https://en.cppreference.com/w/cpp/chrono/duration

```c++
using namespace std::chrono_literals;

const auto duration = 2h;
const auto duration = 45min;
const auto duration = 1s;
const auto duration = 16ms;
const auto duration = 1.0h + 53min;
```

Но user-defined literals в стандартной библиотеке используются не только для задания единиц измерения, но и для строк:

```c++
const auto str1 = "hello c++";    // const char*
const auto str2 = "hello c++"s;   // std::string
const auto str3 = "hello c++"sv;  // std::string_view
```

<br />

##### C++11: static assertions / C++17 static assertions without message

https://en.cppreference.com/w/cpp/language/static_assert

> Performs compile-time assertion checking

`static_assert(condition, message)` - compile-time проверка условий. Если проверка не проходит, компиляция останавливается с ошибкой и выдаёт сообщение `message`. C С++17 `message` опционально, его можно опускать.

Примеры использования:

```c++
#include <type_traits>

// хотим разрешить создавать Point<T> только
// для целых и вещественных типов T:
template <typename T>
struct Point
{
    static_assert(std::is_floating_point<T>::value || std::is_intergal<T>::value,
                  "Point<T> must be created only for floating or integer types");
    
    T x;
    T y;
    T z;
};

// хотим сделать свой swap для ограниченного набора типов,
// чтобы swap гарантированно не бросал исключения
template <typename T>
void swap(T& a, T& b) noexcept
{
    static_assert(std::is_copy_constructible<T>::value &&
                  std::is_nothrow_copy_constructible<T>::value &&
                  std::is_nothrow_copy_assignable<T>::value);

    auto t = b;
    b = a;
    a = t;
}
// Вопрос: кто-нибудь помнит, как мы делали swap через SFINAE и зачем?

// например, если код ниже содержит трюки, работающие
// только для 64-битных указателей, можно добавить compile-time проверку:
static_assert(sizeof(void*) == 8);
```

<br />

##### Атрибуты: [[maybe_unused]], [[nodiscard]], [[deprecated]], [[fallthrough]], [[likely]], [[unlikely]] и др.

Полный список атрибутов:

https://en.cppreference.com/w/cpp/language/attributes

<br />

##### [[fallthrough]] (since C++17)

https://en.cppreference.com/w/cpp/language/attributes/fallthrough

В таком коде компилятор должен кинуть warning, что не все ветки `switch` заканчиваются `break;`.

```c++
switch (message.log_level)
{
    // ignore debug and info logs
    case LogLevel::Debug:
    case LogLevel::Info:
        break;

    // process warning and errors
    case LogLevel::Warning:
    case LogLevel::Error:
        process_log_message(message.content);
        break;
};
```

Это корректный код, но из-за того, что программисты часто делали ошибки, забывая писать `break;`, разработчики компиляторов добавили соответствующий warning.

`[[fallthrough]]` - способ программиста сказать компилятору "да, я уверен, здесь намеренно пропущен `break`".

Код ниже решает аналогичную задачу, но без warning-ов:


```c++
switch (message.log_level)
{
    // ignore debug and info logs
    case LogLevel::Debug:
        [[fallthrough]];
    case LogLevel::Info:
        break;

    // process warning and errors
    case LogLevel::Warning:
        [[fallthrough]];
    case LogLevel::Error:
        process_log_message(message.content);
        break;
};
```

`[[fallthrough]]`, за которым по графу выполнения не идёт другого `case`/`default` - ill-formed-выражение. Скорее всего, не скомпилируется.

Более интересный пример:

```c++
switch (message.log_level)
{
    // ignore debug and info logs
    case LogLevel::Debug:
    case LogLevel::Info:
         if (message.important)
            [[fallthrough]];
         else
            break;

    // process warning and errors
    case LogLevel::Warning:
        [[fallthrough]];
    case LogLevel::Error:
        process_log_message(message.content);
        break;
};
```

<br />

##### [[deprecated]] (since C++14)

https://en.cppreference.com/w/cpp/language/attributes/deprecated

`[[deprecated]]` - сравнительно мягкий способ избавления от устаревшего публичного кода. Использование сущностей, помеченных как `deprecated`, генерирует warning.

План избавления от устаревшего кода:
1. Рассказываем всем, что в будущем планируем этот код удалить (и говорим, когда)
2. Помечаем его `[[deprecated]]` - у пользователей кода (других программистов) начинают сыпаться warning-и
3. Надеемся, что они эти warning-и в течение года-двух таки пофиксят
4. Удаляем устаревший код
5. У тех, кто не пофиксил warning-и, сломалась компиляция (но у них хотя бы было время, и компилятор их предупреждал)

```c++
class Player
{
public:
    [[deprecated("players import is going to be removed in release 15.4")]]
    Player(int id, const std::string& nickname, bool is_imported);

    Player(int id, const std::string& nickname);
    
    ...
    
    [[deprecated("players import is going to be removed in release 15.4")]]
    bool is_imported() const noexcept;
    
    ...
};
```

```c++
[[deprecated("players import is going to be removed in release 15.4")]]
Player import_player();
```

В качестве байки: другой способ за-deprecate-ить код - sleeping constructor pattern.

Что можно пометить `[[deprecated]]`:
* `class/struct/union`: `struct [[deprecated]] S;`
* `typedef/using`:  `[[deprecated]] typedef S* PS;`, `using PS [[deprecated]] = S*;`
* переменные (глобальные, static data): `[[deprecated]] int x;`
* поле `class/struct/union`: `union U { [[deprecated]] int n; };`
* функцию: `[[deprecated]] void f();`
* `namespace`: `namespace [[deprecated]] NS { ... }`
* enumeration: `enum [[deprecated]] E {};`
* enumerator: `enum { A [[deprecated]], B [[deprecated]] = 42 };`
* template specialization: `template<> struct [[deprecated]] X<int> {};`

<br />

##### [[maybe_unused]] (since C++17)

https://en.cppreference.com/w/cpp/language/attributes/maybe_unused

`[[maybe_unused]]` - подсказка компилятору, что сущность может быть неиспользуемой, и это нормально. Подавляет warning-и о таких сущностях.

**Пример**: неиспользуемый аргумент

```c++
class Animal
{
public:
    virtual void say(bool loudly) = 0;
};

class Fish : public Animal
{
public:
    void say([[maybe_unused]] bool loudly) override {}
};
```

**Пример**: warning зависит от режима компиляции (debug/release)

```c++
std::set<std::string> cities = ...;
[[maybe_unused]] const auto erased_count = cities.erase("Carthage");
assert(erased_count == 1);
```

Ещё `[[maybe_unused]]` можно навешивать на классы, поля класса, глобальные переменные, функции, `typedef`-ы, но практической пользы в этом мало.

<br />

##### [[nodiscard]] (since C++17)

`[[nodiscard]]` - запрет программисту игнорировать возвращаемое значение функции.

```c++
[[nodiscard]] int sqr(int x) {
    return x * x;
}
```

Использование:

```c++
int nine = sqr(3); // OK

sqr(2); // compilation warning
```

<br />

Рекомендации по использованию:
1. _Рекомендаций от сообщества нет_
2. Стоит помечать функции `[[nodiscard]]`, если вы уверены, что игнорирование возрвщаемого значения выглядит как неестественное использование или ошибка

**Пример 1**: когда `nodiscard` поможет отловить ошибки

```c++
// класс immutable множества
class set
{
    ...

    // return the new set having this with |value| added
    [[nodiscard]] set add(int value);
};

set s = {1, 2, 3};
s.add(4);  // warning
```

**Пример 2**: когда `nodiscard` поможет отловить ошибки

```c++
// класс динамического массива из стандартной библиотеки
class vector
{
    ...
    // пуст ли массив
    [[nodiscard]] bool empty();
};

// класс динамического массива из Unreal Engine 4
class TArray
{
    ...
    // очистить
    void Empty();
};

v.empty();  // чего хотел программист Unreal Engine 4? Проверить на пустоту или очистить массив?
            // warning: unused return value
```

__Замечание__: `std::vector::empty` помечен `[[nodiscard]]` начиная с С++20

**Пример 3**: когда `nodiscard` носит скорее рекомендательных характер - поможет отловить мелкие глупости

```c++
class Person
{
public:
    [[nodiscard]]
    Person(string name, string surname);
    
    [[nodiscard]]
    const string& name() const noexcept { return name_; }
    
    [[nodiscard]]
    const string& surname() const noexcept { return surname_; }

private:
    string name_;
    string surname_;
};


Person("Dobrynia", "Nikitich");  // warning

Person p{"Dobrynia", "Nikitich"};
p.name();  // warning

```

**Пример 4**: когда `nodiscard` поможет отловить проблемы лишних вычислений (для performance)

```c++
[[nodiscard]]
string build_full_name(const Person& p)
{
    return p.name + ' ' + p.surname;
}


Person p{"Dobrynia", "Nikittich"};
build_full_name(p);  // warning
```

**Пример 5**: когда `nodiscard` вреден

```c++
// return success / failure indicator
bool send_logs_to_server(const string& logs_folder);
```

Не всегда программист обязан обрабатывать ошибки алгоритмов.

Например, для функции отправки логов на сервер:
* production код может игнорировать факт ошибки
* unit tests код может использовать метку об ошибке для тестирования 

<br />

##### [[likely]], [[unlinkely]] (since C++20)

https://en.cppreference.com/w/cpp/language/attributes/likely

`[[likely]], [[unlinkely]]` - стандартизированный способ сказать компилятору, что ветка `if`/`switch` будет выполняться с большей/меньшей вероятностью.

```c++
void game_loop()
{
    while (true)
    {
        auto& player_controller = get_player_controller();
        
        [[unlikely]] if (player_controller.is_player_dead())
        {
            finish_game();
            return;
        }

        ...
    }
}
```

**Вопрос**: зачем их ввели?

<br />

#### C++17 string_view

https://en.cppreference.com/w/cpp/string/basic_string_view

`std::string_view` - невладеющая обёртка над массивом символов. Содержит указатель на начало и размер.

**Вопросы**:
* чему равен `sizeof(std::string_view)`?
* как `std::string_view` передавать в функции?

<br />

`std::string_view` ввели чтобы решить проблему перегруженности интерфейсов.

Например, выдуманная библиотечная функция `int convert_to_int(<строка>)` для удобства использования должна иметь 3 перегрузки:

```c++
// 1
int convert_to_int(const std::string& s);

// 2
int convert_to_int(const char* s);

// 3
int convert_to_int(const char* s, std::size_t n);
```

**Вопрос:**
* чем вариант 3 отличается от 2?
* в каких случаях можно пользоваться только вариантом 3, а вариант 2 не сработает?
* в каких случаях вариант 2 удобнее и быстрее чем варинт 3?

<br />

Варианты вызова `convert_to_int`:

```c++
// 1
std::string s = read_user_input();
int x = convert_to_int(s);

// 2
convert_to_int("123");

// 3
std::string json = read_json();  // и как-то мы знаем, что с 12-го по 16-ый лежит число
convert_to_int(&json[12], 5);
```

<br />

`std::string_view` позволяет сохранить все возможные варианты вызова, но оставить один интерфейс:

```c++
int convert_to_int(std::string_view sv);
```

тогда:

```c++
// 1
//
// создаётся временный std::string_view, который инициализируется
// адресом s.c_str() и размером s.size().
//
// std::string_view корректен, пока жив s. Обазянность за отслеживанием
// времени жизни массива, на который ссылается std::string_view, лежит
// на программисте.
std::string s = read_user_input();
int x = convert_to_int(s);

// 2
//
// создаётся временный std::string_view, который инициализируется
// адресом s и размером strlen(s) - в момент создания конструктор
// std::string_view будет узнавать длину строки (!)
convert_to_int("123");

// 3
//
// явно указываем, на какой массив ссылается std::string_view
std::string json = read_json();  // и как-то мы знаем, что с 12-го по 16-ый лежит число
convert_to_int(std::string_view{&json[12], 5});
```

<br />

**Замечание:** свойство, что `std::string_view` обязан хранить длину может быть как полезным, так и вредным:

вот такой код будет работать медленнее чем мог бы на обычном `const char *` (почему?):

```c++
bool is_male(const std::string_view fullname)
{
    return fullname.starts_with("Mr.");
}

is_male("Mr. Bond. James Bond.");
is_male("Mr. Antoine Marie Jean-Baptiste Roger, comte de Saint-Exupéry");
```

<br />

А такой код через `std::string_view` может работать быстрее (почему?):

```c++
std::stringstream ss;
for (int i = 10; i != 1; --i)
    ss << i << " green bottles hanging on the wall. And if one green bottle should accidentally fall...\n";
```

```c++
constexpr std::string_view msg =
    " green bottles hanging on the wall. And if one green bottle should accidentally fall...\n";
std::stringstream ss;
for (int i = 10; i != 1; --i)
    ss << i << msg;
```

<br />

##### std::span (since C++20)

https://en.cppreference.com/w/cpp/container/span

Идея, стоящая за `std::span` аналогична `std::string_view` - невладеющий view на массив (или часть массива).

С `std::span` комитет пошёл чуть дальше, и добавил возможности для оптимизаций, когда размер данных известен во время компиляции.

Пример использования:

```c++
double sum(const std::span<double> v)
{
    double rv = 0.0;
    for (double x : v)
        rv += x;
    return rv;
}
```

За оптимизации для известных размеров отвечает второй шаблонный параметр:

```c++
template< 
    class T, 
    std::size_t Extent = std::dynamic_extent
> class span;
```

**Вопрос на понимание**:

* чему равен `sizeof(std::span<double>)`?
* чему равен `sizeof(std::span<double, 5>)`?

<br />

##### heterogeneous lookup (ordered map/set since C++14, unordered map/set since C++20)

https://www.bfilipek.com/2019/05/heterogeneous-lookup-cpp14.html

http://www.open-std.org/jtc1/sc22/wg21/docs/papers/2018/p0919r2.html

Сначала рассмотрим мотивацию для новой фичи:

```c++
std::map<std::string, int> name_to_age = {
    {"Ilya Muromec", 42},
    {"Dobrynia Nikitich", 57},
    {"Alesha Popovich", 23}
};

if (name_to_age.find("Koshei Bessmertniy") == name_to_age.end())
    std::cout << "how somebody can be too old?" << std::endl;
```

**Вопрос**: В этом примере ошибок нет (вроде бы), но какая проблема в нём скрыта?

<details>
<summary>ответ</summary>
<details>
<p>

Чтобы поискать значение в `std::map` создаётся промежуточный объект `std::string`. В случае, если он не влезает в SSO, это дорого: нужна аллокация.

</p>
</details>
</details>

<br />

heterogeneous lookup - трюк, который чинит проблему:

```c++
std::map<std::string, int>              name_to_age; // контейнер со стандартным поиском
std::map<std::string, int, std::less<>> name_to_age; // контейнер с "гетерогенным" поиском
```

<br />

Трюк достаточно просто использовать, но как он работает?

Строчки выше компилятор разворачивает в следующее:

```c++
std::map<std::string, int> std::less<std::string>> name_to_age; // контейнер со стандартным поиском
std::map<std::string, int, std::less<>>            name_to_age; // контейнер с "гетерогенным" поиском
```

Классы-компараторы в стандартной библиотеке выглядят так (примерно и упрощённо для иллюстарции идеи, на самом деле всё много сложнее):

```c++
// для конкретного типа (в том числе std::string)
template<typename T>
struct less {
    bool operator < (const T& lhs, const T& rhs) const {
        return lhs < rhs;
    }
};

// специализация для T=void
template<>
struct less<void> {
    using is_transparent = void;
    
    template<typename T1, typename T2>
    bool operator < (const T1& lhs, const T2& rhs) const {
        return lhs < rhs;
    }
};
```

**Вопрос**: в каком месте требуется конструирование `std::string` для стандартного поиска, и почему можно его избежать в гетерогенном варианте?

Осталось дело за малым - скомпилировать transparent-вариант поиска, если он доступен с переданными типами аргументов.

В libc++ это делается так:

```c++
iterator find(const key_type& __k)             {return __tree_.find(__k);}

template <typename _K2>
typename enable_if<__is_transparent<_Compare, _K2>::value, iterator>::type
find(const _K2& __k)                           {return __tree_.find(__k);}
```

В listdc++ так:

```c++
iterator find(const key_type& __x) { return iterator(_Base::find(__x), this); }

template<typename _Kt,
         typename _Req = typename __has_is_transparent<_Compare, _Kt>::type>
iterator find(const _Kt& __x)      { return { _Base::find(__x), this }; }
```

**Вопрос**: как это работает?

<br />

##### init-if/switch (since C++ 17)

Синтаксический сахар для сокращения кода:

```c++
if (init-statement; condition)
    ...
else
    ...
```

До С++17:

```c++
auto& network_system = NetworkSystem::get(app_context);
if (network_system->is_enabled())
    network_system->post(logs_data);
```

После С++17:

```c++
if (auto& network_system = NetworkSystem::get(app_context); network_system->is_enabled())
    network_system->post(logs_data);
```

Переменная, инициализированная в `init-statement`, доступна и в `else`-блоке:

```c++
if (auto& network_system = NetworkSystem::get(app_context); network_system->is_enabled())
    network_system->post(logs_data);
else
    std::cout << "network is disabled, reason: " << network_system->disabled_reason() << std::endl;
```

<br />

##### slicing ordered/unordered map/set (since C++17)

Рассмотрим пример - мотивацию.

Задача - перекладывать элементы из одного отображения в другое в процессе обработки. Нас интересует только процесс перекладывания. Псевдокод:

```c++
std::map<int, std::string> id_to_name = /*...*/;  // набор элементов для обработки
std::map<int, std::string> id_to_name_processed;  // уже обработанные элементы

while (true)
{
    const int id = take_id_to_process();
    const bool ok = process_item(id);
    if (!ok)
        break;
    
    // HERE: transfer from unprocessed to processed
    id_to_name_processed[id] = id_to_name[id];
    id_to_name.remove(id);
}
```

**Вопрос**: как устроен `std::map` внутри? какая работа будет происходить в процессе переноса?

<br />

Можно ускорить процесс переноса элементов между map/set одинакового типа, вычленяя из них сами ноды и вставляя ноды в результат:

Было:

```c++
id_to_name_processed[id] = id_to_name[id];
id_to_name.remove(id);
```

Стало:

```c++
id_to_name_processed.insert(
    id_to_name.extract(id));
```

Или так:

```c++
id_to_name_processed.insert(
    id_to_name.extract(
        id_to_name.find(id)));
```

<br />

Другой рабочий вариант slicing - алгоритм `merge`, который внутри заоптимизирован на перекидывание нод контейнера:

```c++
std::set<int> src = {1, 3, 5};
std::set<int> dst = {2, 4, 5};
dst.merge(src);
// src == { 5 }
// dst == { 1, 2, 3, 4, 5 }
```

Пример использования - три множества людей соберём в одно:

```c++
std::set<Person> regiment_l = get_left_hand_regiment();
std::set<Person> regiment_c = get_big_regiment();
std::set<Person> regiment_r = get_right_hand_regiment();

std::set<Person> army = std::move(regiment_l);
army.merge(regiment_c);
army.merge(regiment_r);
```

<br />

##### std::optional (since C++17)

https://en.cppreference.com/w/cpp/utility/optional

`std::optional<T>` - класс для хранения опционально присутствюзего значения.

Основные функции:
* можно спросить `has_value()` - есть ли значение
* можно запросить само значение через `get()` или `operator->()`
* в своём деструкторе позовёт деструктор у значения, если оно есть
* значение хранится в ляуте класса (т.е. сам `std::optional` свободен от аллокаций)

Очень простой и очень полезный класс.

Сценарии применения:

1. возврат результата из функции, когда результат может отсутствовать:

```c++
std::optional<Point3D>  get_intersection(Line3D l1, Line3D l2);
```

2. опциональное поле у класса либо поле с ленивой инициализацией

```c++
class SalesmanWidget
{
    ...

    // optional, потому что может быть не запущено задачи
    std::optional<ProblemHandler> handler;  // не требует аллокаций в отличие от std::unique_ptr
};
```

Как НЕ надо применять `std::optional`:
            
1. `std::optional<bool>` - см. `boost::tribool`
2. как опциональный аргумент:

```c++
void func(std::optional<int> arg);
```

**Вопрос**: а почему так не надо?

**Вопрос**: чему равен `sizeof(std::optional<int>)`?

<br />

##### structured bindings (since C++17)

structured bindings - синтаксический сахар - автоматическая распаковка структур и кортежей по именам.

Сравните насколько structured bindings украшают итерирование по отображениям:

```c++
std::unordered_map<int, string> id_to_name = /* ... */;

// before C++17
for (const auto& id_and_name : id_to_name)
{
    const auto& id = id_and_name.first;
    const auto& name = id_and_name.second;
    /* process id and name */
}
    
// C++17:
for (const auto& [id, name]: id_to_name)
{
    /* process id and name */
}
```

Или как structured bindings позволяют распаковать структуру:

```c++
struct Point
{
    float x, y, z;
};

std::vector<Point> points = /* ... */;

// before C++17:
for (const auto p : points)
    std::cout << p.x * p.x + p.y * p.y + p.z * p.z << std::endl;

// C++17:
for (const auto [x, y, z] : points)
    std::cout << x * x + y * y + z * z << std::endl;
```

Или так:

```c++
struct Person
{
    std::string name;
    std::string surname;
    int age;
};

// before C++17:
const Person person = get_friend();
std::cout << person.name + " " + person.surname << std::endl;

// C++17:
const auto [name, surname, _] = get_friend();
std::cout << name + " " + surname << std::endl;
```

<br />

Как делать structured bindings доступными для классов с приватными полями - было на лекции по `constexpr`.

<br />

Давайте посмотрим, что происходит внутри structured binding:

**Вопрос-наброс**: может ли компилятор в этом примере применить NRVO-оптимизацию?

```c++
struct Person
{
    std::string name;
    std::string surname;
    int age;
};

Person get_friend();

std::string get_friend_name()
{
    auto [name, surname, age] = get_friend();
    return name;   // <---------------------------- тут
}
```

Нет, компилятор скорее всего не сможет здесь применить NRVO. Когда работает structured bindings, создаётся полноценный объект `Person`, а затем создаются alias-ы на его поля. В `return name` для компилятора происходит доступ до поля структуры, он не может это поле разместить отдельно от структуры.

<br />

##### using enum (C++ 20)

http://www.open-std.org/jtc1/sc22/wg21/docs/papers/2019/p1099r4.html

using enum - синтаксический сахар для сокращения кода, чтобы поменьше писать имя `enum`.

Заведём enum с ужасающе длинным названием:

```c++
enum class rgb_color_channel {
    red,
    green,
    blue
};
```

Напишем `switch` по нему в старом стиле:

```c++
// before C++20
std::string_view to_string(rgba_color_channel channel) {
  switch (channel) {
    case rgb_color_channel::red:   return "red";
    case rgb_color_channel::green: return "green";
    case rgb_color_channel::blue:  return "blue";
  }
}
```

И в новом стиле:

```c++
// after C++20
std::string_view to_string(rgba_color_channel channel) {
  switch (my_channel) {
    using enum rgb_color_channel;  // <- !!!
    case red:   return "red";
    case green: return "green";
    case blue:  return "blue";
  }
}
```

using enum можно использовать и в других местах:

```c++
// before C++20
if (channel == rgb_color_channel::red)
    std::cout << "red";
```

```c++
// after C++ 20
using enum rgb_color_channel;
if (channel == red)
    std::cout << "red";
```

<br />

**Прочие мелкие улучшения, не вошедшие в лекцию**:

* [C++11: inline namespaces](https://github.com/AnthonyCalandra/modern-cpp-features#inline-namespaces)
* [C++11: ref-qualified member functions](https://github.com/AnthonyCalandra/modern-cpp-features#ref-qualified-member-functions)
* [C++14: variable templates](https://github.com/AnthonyCalandra/modern-cpp-features#variable-templates)
* [C++17: std::filesystem](https://en.cppreference.com/w/cpp/filesystem)
* [C++17: inline variables](https://github.com/AnthonyCalandra/modern-cpp-features#inline-variables)
* [C++17: nested namespaces](https://github.com/AnthonyCalandra/modern-cpp-features#nested-namespaces)
* [C++20: spaceship operator](http://www.open-std.org/jtc1/sc22/wg21/docs/papers/2019/p1614r1.html)
* [C++20: calendar + chrono improvements](https://en.cppreference.com/w/cpp/chrono)
* [C++20: synchronization library improvements](http://www.open-std.org/jtc1/sc22/wg21/docs/papers/2019/p1135r5.html)